# Intuitional Description

The LD score of a variant is a measure to describe how connected or tagged this variant is with other variants by summing up the squared correlation (r²) with every other variant under consideration.


# Graphical Summary

![fig](./cartoons/linkage_disequilibrium_score.svg)

# Key Formula

The LD score for a SNP is the sum of LD $r^2$ measured with all other SNPs:

$$
l_j = \sum_{k=1, k \neq j}^M r^2(\textbf{X}_j, \textbf{X}_k)
$$

# Technical Details

The **LD score** is a measure of the extent to which a given variant is in linkage disequilibrium (LD) with other variants across the genome. It is used to summarize the amount of genetic information (in terms of LD) that a variant shares with all other variants in a region of interest. The LD score for a variant $j$ is defined as the sum of the squared correlation coefficients $r^2$ between that variant and all other variants in the genome, typically within a specified genomic window or region. Mathematically, the LD score for variant $j$ is given by:

$$
l_j = \sum_{k=1, k \neq j}^M r^2_{i,k}
$$

<!-- Since $\textbf{X}$ is standardized, we can just calculate the sum of the squared sample correlations like this:

$$\widetilde{l}_{j} = \frac{1}{N^2}\textbf{X}^\top_j\textbf{X}\textbf{X}^\top\textbf{X}_j - 1$$ -->
However, this is not an unbiased estimate. We can correct for the bias like this:

$$
r_\text{adj}^2 =  \hat{r}^2 - \frac{1-\hat{r}^2}{N-2}
$$
$$
l_j = \sum_{k=1, k \neq j}^M  r_\text{adj}^2 
$$

This score reflects how much a variant is correlated with other variants across the genome, providing insight into the local structure of LD around that variant.



# Example

This example demonstrates the process of calculating Linkage Disequilibrium (LD) scores from a genotype matrix. It computes the correlation matrix based on the standardized genotype matrix, transforms it to squared correlations, and calculates raw LD scores. A bias correction is then applied to adjust for finite sample size effects, resulting in corrected LD scores that provide a measure of genetic variant connectivity.

In [33]:
# Clear the environment
rm(list = ls())

# Define genotypes for 5 individuals at 3 variants
# These represent actual alleles at each position
# For example, Individual 1 has genotypes: CC, CT, AT
genotypes <- c(
 "CC", "CT", "AT",  # Individual 1
 "TT", "TT", "AA",  # Individual 2
 "CT", "CT", "AA",  # Individual 3
 "CC", "TT", "AA",  # Individual 4
 "CC", "CC", "TT"   # Individual 5
)
# Reshape into a matrix
geno_matrix <- matrix(genotypes, nrow=5, ncol=3, byrow=TRUE)
rownames(geno_matrix) <- paste("Individual", 1:5)
colnames(geno_matrix) <- paste("Variant", 1:3)

alt_alleles <- c("T", "C", "T")

# Convert to raw genotype matrix using the additive / dominant / recessive model
Xraw_additive <- matrix(0, nrow=nrow(geno_matrix), ncol=ncol(geno_matrix)) # dount number of non-reference alleles

rownames(Xraw_additive) <- rownames(geno_matrix)
colnames(Xraw_additive) <- colnames(geno_matrix)

for (i in 1:nrow(geno_matrix)) {
  for (j in 1:ncol(geno_matrix)) {
    alleles <- strsplit(geno_matrix[i,j], "")[[1]]
    Xraw_additive[i,j] <- sum(alleles == alt_alleles[j])
  }
}
X <- scale(Xraw_additive, center=TRUE, scale=TRUE)

# calculate the LD
R = cor(X)

In [34]:
cat("The LD matrix is:\n")
R

The LD matrix is:


,Variant 1,Variant 2,Variant 3
Variant 1,1.0000000,-0.4677072,-0.562500
Variant 2,-0.4677072,1.0000000,0.868599
Variant 3,-0.5625000,0.8685990,1.000000


In [35]:
# Calculate the squared correlation matrix to represent pairwise variant correlations
# R^2 provides the proportion of variance explained by each pair of variants
R_squared <- R^2  # Element-wise square to get r^2 (squared correlation coefficients)

# Get sample size and number of variants
N <- nrow(X)  # Number of individuals/samples
M <- ncol(X)  # Number of genetic variants

# Calculate raw LD scores:
# 1. Sum squared correlations across each column
# 2. Subtract the diagonal (variant's correlation with itself)
# This gives the total genetic connectivity for each variant
cat("The original LD scores are:\n")
ld_scores_raw <- colSums(R_squared) - diag(R_squared)
ld_scores_raw


The original LD scores are:


Variant 1 Variant 2 Variant 3 
0.5351562 0.9732143 1.0708705

In [37]:
# First calculate the adjusted R² values for each pair
R_squared_adj <- R_squared  # Create a copy to modify

# Apply the adjustment formula to each non-diagonal element
for (i in 1:M) {
  for (j in 1:M) {
#   Apply the adjustment formula: r²_adj = r² - (1-r²)/(N-2)
    R_squared_adj[i, j] <- R_squared[i, j] - (1 - R_squared[i, j]) / (N - 2)
  }
}
# Calculate adjusted LD scores by summing the adjusted r² values
cat("The adjusted LD scores are:\n")
ld_scores_adjusted <- colSums(R_squared_adj) - diag(R_squared_adj)
ld_scores_adjusted

The adjusted LD scores are:


Variant 1 Variant 2 Variant 3 
0.0468750 0.6309524 0.7611607